In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from finance_byu.summarize import summary
import pyarrow
# !pip uninstall pandas
# !pip install pandas==2.2.3
print(pd.__version__)

load_dotenv()  # reads .env file
SAVE_PATH = os.getenv("SAVE_PATH")

2.2.3


In [2]:
df = pd.read_feather(SAVE_PATH)

df = df[(df.date >= pd.to_datetime('1980-01-01'))]

# df = df.loc[~df['cash'].isna()]
# df = df.loc[~df['current'].isna()]
# df['ret'] = df['ret']/100

print(df)
df.describe()

         permno       date     cusip ticker  shrcd  excd   siccd        prc  \
0         10000 1986-02-28  68391610  OMFGA   10.0   3.0  3990.0   -3.25000   
1         10000 1986-03-31  68391610  OMFGA   10.0   3.0  3990.0   -4.43750   
2         10000 1986-04-30  68391610  OMFGA   10.0   3.0  3990.0   -4.00000   
3         10000 1986-05-30  68391610  OMFGA   10.0   3.0  3990.0   -3.10938   
4         10000 1986-06-30  68391610  OMFGA   10.0   3.0  3990.0   -3.09375   
...         ...        ...       ...    ...    ...   ...     ...        ...   
4963798   93436 2024-08-30  88160R10   TSLA   11.0   3.0  9999.0  214.11000   
4963799   93436 2024-09-30  88160R10   TSLA   11.0   3.0  9999.0  261.63000   
4963800   93436 2024-10-31  88160R10   TSLA   11.0   3.0  9999.0  249.85001   
4963801   93436 2024-11-29  88160R10   TSLA   11.0   3.0  9999.0  345.16000   
4963802   93436 2024-12-31  88160R10   TSLA   11.0   3.0  9999.0  403.84000   

              ret         vol  ...  fqtr  popsrc   

,permno,date,shrcd,excd,siccd,prc,ret,vol,shr,cumfacshr,GVKEY,fyearq,fqtr,current,assets,cash,mkvaltq
count,3.884214e+06,3884214,3.884214e+06,3.884214e+06,3.884214e+06,3.884214e+06,3.884214e+06,3.789998e+06,3.884214e+06,3.884214e+06,771716.000000,771716.000000,771716.000000,2.513126e+06,3.004325e+06,1.013161e+06,2.411260e+05
mean,5.805571e+04,2003-07-22 15:46:36.439742208,2.003587e+01,2.370074e+00,5.299302e+03,3.868436e+01,9.640434e-03,1.230991e+05,6.156274e+04,1.595491e+00,43922.336861,2001.669837,2.498897,1.059314e+03,8.024711e+03,6.311959e+02,6.438926e+03
min,1.000000e+04,1980-01-31 00:00:00,1.000000e+01,1.000000e+00,0.000000e+00,-1.832500e+03,-9.958300e-01,0.000000e+00,0.000000e+00,0.000000e+00,1001.000000,1980.000000,1.000000,-6.000000e-02,-6.333300e+01,-9.174000e+00,0.000000e+00
25%,2.370300e+04,1993-01-29 00:00:00,1.100000e+01,1.000000e+00,3.599000e+03,3.600000e+00,-5.837400e-02,1.692000e+03,4.507000e+03,1.000000e+00,8475.000000,1992.000000,2.000000,1.882700e+01,4.985000e+01,1.193300e+01,1.261875e+02
50%,7.188800e+04,2002-12-31 00:00:00,1.100000e+01,3.000000e+00,5.961000e+03,1.301000e+01,1.111000e-03,9.073000e+03,1.330000e+04,1.000000e+00,17140.000000,2000.000000,3.000000,8.667900e+01,2.814660e+02,5.626300e+01,5.833356e+02
75%,8.402300e+04,2014-12-31 00:00:00,1.200000e+01,3.000000e+00,6.726000e+03,2.750000e+01,6.137800e-02,5.068200e+04,4.092400e+04,1.000000e+00,61519.000000,2011.000000,3.000000,3.987480e+02,1.653510e+03,2.340000e+02,2.660050e+03
max,9.343600e+04,2024-12-31 00:00:00,7.500000e+01,3.300000e+01,9.999000e+03,7.240400e+05,3.900000e+01,2.012427e+08,2.920640e+07,7.680000e+02,356128.000000,2025.000000,4.000000,3.438409e+05,4.210048e+06,3.141480e+05,3.766500e+06
std,2.967187e+04,NaN,1.997851e+01,1.051395e+00,2.222349e+03,2.308299e+03,1.813386e-01,8.106230e+05,2.757156e+05,4.823444e+00,57460.374378,12.059066,1.114667,5.519996e+03,7.679843e+04,4.168667e+03,4.459374e+04


In [3]:
df['logret'] = np.log(1 + df['ret'])
df['mom'] = df.groupby('permno')['logret'].rolling(11,11).sum().reset_index(drop=True)
df['mom'] = df.groupby('permno')['mom'].shift(2)
df['me'] = df['shr'] * df['prc']


In [4]:
df['prclag'] = df.groupby('permno')['prc'].shift()
df['melag'] = df.groupby('permno')['me'].shift()
# df['cashtoasset'] = df["cash"] / df["assets"]
df['currenttoasset'] = df["current"] / df["assets"]
df.drop(columns=["cash", "current", "assets"])
# df = df.loc[df.date >= pd.to_datetime("1990-01-01")]

df = df.query("mom == mom and prclag >= 5").reset_index(drop=True)

In [5]:
df.describe()

,permno,date,shrcd,excd,siccd,prc,ret,vol,shr,cumfacshr,...,current,assets,cash,mkvaltq,logret,mom,me,prclag,melag,currenttoasset
count,1.742570e+06,1742570,1.742570e+06,1.742570e+06,1.742570e+06,1.742570e+06,1.742570e+06,1.742518e+06,1.742570e+06,1.742570e+06,...,1.142367e+06,1.381591e+06,456345.000000,1.027440e+05,1.742570e+06,1.742570e+06,1.742570e+06,1.742570e+06,1.742570e+06,1.142365e+06
mean,4.640521e+04,2003-07-21 06:48:25.278525440,1.981356e+01,2.117167e+00,5.330223e+03,5.236952e+01,9.379245e-03,1.261848e+05,7.991602e+04,1.823152e+00,...,1.456175e+03,9.745466e+03,715.218453,9.613979e+03,2.053613e-03,-3.118547e-02,3.609569e+06,5.324372e+01,3.588622e+06,4.733545e-01
min,1.000100e+04,1980-03-31 00:00:00,1.000000e+01,1.000000e+00,0.000000e+00,-1.010535e+03,-9.812950e-01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,-9.174000,0.000000e+00,-3.978964e+00,-1.217856e+01,-1.300461e+08,5.000000e+00,0.000000e+00,0.000000e+00
25%,1.822375e+04,1993-02-26 00:00:00,1.100000e+01,1.000000e+00,3.599000e+03,1.100000e+01,-4.488300e-02,2.365000e+03,5.757000e+03,1.000000e+00,...,5.312000e+01,1.414550e+02,19.313000,3.839272e+02,-4.592143e-02,-2.184672e-01,8.061579e+04,1.150000e+01,8.289000e+04,2.604481e-01
50%,4.749000e+04,2001-03-30 00:00:00,1.100000e+01,2.000000e+00,5.812000e+03,1.968750e+01,5.829000e-03,1.100800e+04,1.603000e+04,1.000000e+00,...,1.883540e+02,6.192110e+02,86.299000,1.361574e+03,5.812077e-03,4.082356e-02,2.921174e+05,2.001000e+01,2.919173e+05,4.786360e-01
75%,7.608100e+04,2016-03-31 00:00:00,1.200000e+01,3.000000e+00,6.726000e+03,3.387500e+01,5.806000e-02,5.748400e+04,4.926000e+04,1.000000e+00,...,7.172650e+02,2.918814e+03,325.150000,4.776080e+03,5.643704e-02,2.416110e-01,1.285626e+06,3.400000e+01,1.276078e+06,6.753913e-01
max,8.276600e+04,2024-12-31 00:00:00,7.500000e+01,3.300000e+01,9.999000e+03,6.344400e+05,1.988359e+01,1.707594e+08,2.288030e+07,7.680000e+02,...,3.438409e+05,4.210048e+06,314148.000000,3.766500e+06,3.038964e+00,4.600875e+00,3.785304e+09,6.344400e+05,3.587438e+09,6.319211e+00
std,2.685063e+04,NaN,1.980810e+01,1.119054e+00,2.140739e+03,2.608805e+03,1.234380e-01,6.731155e+05,3.460982e+05,5.273858e+00,...,6.213742e+03,7.943973e+04,3851.958037,5.389681e+04,1.225086e-01,5.572134e-01,2.646658e+07,2.593497e+03,2.618076e+07,2.567789e-01


In [6]:
# df.loc[(df['tic']=='AAPL')&(~df['cash'].isna().index)]

In [7]:
# df = df.loc[~df['cashtoasset'].isna()]
df = df.loc[~df['currenttoasset'].isna()]
df["mom_first"] = df.groupby('date')['mom'].transform(pd.qcut,5,labels=[str(k) for k in range(1, 6)]).shift()
# df["cashtoasset_first"] = df.groupby('date')['cashtoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"]),shift()
# df["currenttoasset_first"] = df.groupby('date')['currenttoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"]).shift()
# df["cashtoasset_second"] = df.groupby(['date', 'mom_first'])['cashtoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"]).shift()
df["currenttoasset_second"] = df.groupby(['date', 'mom_first'])['currenttoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"]).shift()

/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_69304/83352132.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["currenttoasset_second"] = df.groupby(['date', 'mom_first'])['currenttoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"]).shift()


In [8]:
#Equal weighted, cash to asset ratio
port = df.groupby(['date','mom_first','currenttoasset_second'])['ret'].mean()
port = port.unstack(level=['mom_first','currenttoasset_second'])
print('Summary statistics from month to month, mom-currentasset:')
summary(port).round(4)

Summary statistics from month to month, mom-currentasset:


/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_69304/4015121276.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  port = df.groupby(['date','mom_first','currenttoasset_second'])['ret'].mean()


mom_first                     1                                          \
currenttoasset_second         a         b         c         d         e   
count                  535.0000  535.0000  535.0000  535.0000  535.0000   
mean                     0.0095    0.0091    0.0111    0.0101    0.0118   
std                      0.0476    0.0519    0.0568    0.0599    0.0678   
tstat                    4.6187    4.0775    4.5066    3.9076    4.0377   
pval                     0.0000    0.0001    0.0000    0.0001    0.0001   
min                     -0.2518   -0.3031   -0.3099   -0.3004   -0.3167   
25%                     -0.0148   -0.0173   -0.0197   -0.0259   -0.0279   
50%                      0.0124    0.0136    0.0113    0.0144    0.0139   
75%                      0.0375    0.0400    0.0450    0.0468    0.0505   
max                      0.2083    0.1754    0.2006    0.2092    0.2845   

mom_first                     2                                          ...  \
currenttoasset_second         a         b         c         d         e  ...   
count                  535.0000  535.0000  535.0000  535.0000  535.0000  ...   
mean                     0.0099    0.0095    0.0114    0.0105    0.0096  ...   
std                      0.0466    0.0518    0.0546    0.0596    0.0697  ...   
tstat                    4.9170    4.2542    4.8532    4.0623    3.1914  ...   
pval                     0.0000    0.0000    0.0000    0.0001    0.0015  ...   
min                     -0.2457   -0.2885   -0.3010   -0.2982   -0.3387  ...   
25%                     -0.0129   -0.0188   -0.0192   -0.0254   -0.0317  ...   
50%                      0.0143    0.0136    0.0152    0.0125    0.0138  ...   
75%                      0.0359    0.0411    0.0446    0.0483    0.0534  ...   
max                      0.2514    0.1801    0.1704    0.2348    0.2944  ...   

mom_first                     4                                          \
currenttoasset_second         a         b         c         d         e   
count                  535.0000  535.0000  535.0000  535.0000  535.0000   
mean                     0.0088    0.0098    0.0113    0.0111    0.0098   
std                      0.0460    0.0522    0.0555    0.0597    0.0675   
tstat                    4.4416    4.3382    4.7290    4.3106    3.3727   
pval                     0.0000    0.0000    0.0000    0.0000    0.0008   
min                     -0.2324   -0.3024   -0.3025   -0.3238   -0.3165   
25%                     -0.0146   -0.0192   -0.0197   -0.0260   -0.0322   
50%                      0.0119    0.0131    0.0145    0.0153    0.0130   
75%                      0.0341    0.0439    0.0447    0.0487    0.0513   
max                      0.1691    0.1961    0.2000    0.2100    0.2977   

mom_first                     5                                          
currenttoasset_second         a         b         c         d         e  
count                  535.0000  535.0000  535.0000  535.0000  535.0000  
mean                     0.0088    0.0098    0.0112    0.0113    0.0105  
std                      0.0490    0.0531    0.0557    0.0590    0.0667  
tstat                    4.1403    4.2664    4.6670    4.4304    3.6227  
pval                     0.0000    0.0000    0.0000    0.0000    0.0003  
min                     -0.3009   -0.2989   -0.3283   -0.2999   -0.3236  
25%                     -0.0151   -0.0179   -0.0211   -0.0248   -0.0296  
50%                      0.0119    0.0125    0.0143    0.0132    0.0160  
75%                      0.0338    0.0437    0.0440    0.0491    0.0487  
max                      0.3692    0.2038    0.1836    0.2123    0.2345  

[10 rows x 25 columns]

In [9]:
# Equal weighted, cash to asset ratio
port = df.groupby(['date','mom_first'])['ret'].mean()
port = port.unstack(level=['mom_first'])
port['spread'] = port['5'] - port['1']
print('Summary statistics from month to month, mom:')
summary(port).round(4)

Summary statistics from month to month, mom:


/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_69304/1986546384.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  port = df.groupby(['date','mom_first'])['ret'].mean()


mom_first,1,2,3,4,5,spread
count,535.0000,535.0000,535.0000,535.0000,535.0000,535.0000
mean,0.0104,0.0102,0.0105,0.0102,0.0103,-0.0001
std,0.0537,0.0533,0.0531,0.0529,0.0532,0.0096
tstat,4.4696,4.4130,4.5743,4.4708,4.4789,-0.1853
pval,0.0000,0.0000,0.0000,0.0000,0.0000,0.8531
min,-0.2894,-0.2890,-0.2804,-0.2902,-0.2930,-0.0362
25%,-0.0196,-0.0182,-0.0206,-0.0208,-0.0190,-0.0053
50%,0.0130,0.0145,0.0148,0.0141,0.0137,-0.0004
75%,0.0425,0.0437,0.0425,0.0436,0.0428,0.0048
max,0.1856,0.1910,0.1960,0.1891,0.1810,0.0454


In [10]:
# Value weighted:
df['valueReturns'] = df['ret'] * df.groupby('date')['melag'].transform(lambda x: x / x.sum())
port = df.groupby(['date','mom_first'])['valueReturns'].sum()
port = port.unstack(level=['mom_first'])
port['spread'] = port['5'] - port['1']
print('Summary statistics from month to month, mom:')
summary(port).round(4)

Summary statistics from month to month, mom:


/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_69304/919980929.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  port = df.groupby(['date','mom_first'])['valueReturns'].sum()


mom_first,1,2,3,4,5,spread
count,535.0000,535.0000,535.0000,535.0000,535.0000,535.0000
mean,0.0018,0.0023,0.0020,0.0019,0.0019,0.0001
std,0.0087,0.0093,0.0087,0.0087,0.0096,0.0042
tstat,4.8752,5.7328,5.4199,5.1076,4.6723,0.5248
pval,0.0000,0.0000,0.0000,0.0000,0.0000,0.6000
min,-0.0446,-0.0551,-0.0371,-0.0433,-0.0412,-0.0187
25%,-0.0027,-0.0029,-0.0029,-0.0027,-0.0031,-0.0021
50%,0.0026,0.0028,0.0021,0.0026,0.0022,0.0001
75%,0.0075,0.0081,0.0074,0.0069,0.0075,0.0024
max,0.0279,0.0322,0.0354,0.0307,0.0347,0.0176
